In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

 Activating environment at `~/Research/ICRA2020_fast_mpc_in_altro/benchmarks/random_linear_mpc/Project.toml`


# Generate Random Problem

In [2]:
include("random_linear.jl")

# using ControlSystems

n = 12
m = Int(n/2)
N = 10
A,B = gendiscrete(n,m)

Q = Diagonal(10*rand(n))
R = Diagonal(0.1*ones(m))

Qf = Q #dare(A, B, Q, R)

x̄ = rand(n) .+ 1
ū = 0.1*rand(m)
x0 = (rand(n) .- 1) .* x̄ * 0.5
xf = zeros(n)

nothing

# Solve Using Altro

In [3]:
using Altro
using TrajectoryOptimization
using RobotDynamics

const RD = RobotDynamics

dt = 0.1 # doesn't matter, just needs to be non-zero
model = RD.LinearModel(A, B; dt=dt)
objective = LQRObjective(Q, R, Qf, xf, N)

constraints = ConstraintList(n, m, N)
bound = BoundConstraint(n, m, x_min=-x̄, x_max=x̄, u_min=-ū, u_max=ū)
add_constraint!(constraints, bound, 1:N)

tf = (N-1)*dt

problem = Problem(model, objective, xf, tf, x0=x0, integration=RD.PassThrough)
solver = ALTROSolver(problem)
set_options!(solver)
solve!(solver)

b = benchmark_solve!(solver)

@show maximum(b)
@show max_violation(solver)
@show cost(solver)

X = states(solver)
U = controls(solver)

maximum(b) = TrialEstimate(143.271 μs)
max_violation(solver) = 0.0
cost(solver) = 0.5019459876318688


9-element Array{StaticArrays.SArray{Tuple{6},Float64,1,6},1}:
 [-0.17493729475554265, 0.056004594637091176, -0.03972353871387233, 0.059312965664688744, -0.19378115277227123, 0.1281883248834337]
 [-0.00026747947872465816, -0.00723967936982374, -0.043765368687180735, 0.03198451256655593, -0.03154348642423799, -0.0020447516242560193]
 [0.021787673232485733, -0.006325493473086252, -0.0019330984082655267, 0.0028026732337358244, 0.014359177968632203, -0.005370325466506981]
 [0.009698066679932837, 0.0015514062133410583, 0.008487080952806918, 0.0042659080604729605, -0.00019485436799246414, 0.003231802764815106]
 [0.00833768999035514, 0.002271887150393316, 0.00584202354207386, 0.0027702790323874335, 0.0013551864651472345, 0.0017456296821633799]
 [0.0066354800257446, 0.0019026161954529275, 0.004943412948160355, 0.0022570680142218625, 0.0008633729543303381, 0.0014816584951636846]
 [0.00559383619446735, 0.0015734704745267655, 0.004237011868252056, 0.0020736807088530523, 0.00066854155590651, 0.0012

# Solve Using OSQP

In [ ]:
using JuMP
using OSQP
using BenchmarkTools

select(i, n) = (n*(i-1)+1):(n*(i-1)+n)

jump_model = Model(OSQP.Optimizer)
set_silent(jump_model)

@variable(jump_model, x[1:((N)*n)])
@variable(jump_model, u[1:((N-1)*n)])

objective_exp = @expression(jump_model, 0.5*transpose(x[select(N, n)]) * Qf * x[select(N, n)])

@constraint(jump_model, x[select(1, n)] .== x0)

for i=1:N-1
    # dynamics constraints
    @constraint(jump_model, A*x[select(i, n)] + B*u[select(i, m)] .== x[select(i+1, n)])

    # stagewise state cost
    add_to_expression!(objective_exp, 0.5*transpose(x[select(i, n)]) * Q * x[select(i, n)])

    # stagewise control cost
    add_to_expression!(objective_exp, 0.5*transpose(u[select(i, m)]) * R * u[select(i, m)])

    # control/state bound constraints
    @constraint(jump_model, x[select(i, n)] .<= x̄)
    @constraint(jump_model, x[select(i, n)] .>= -x̄)
    @constraint(jump_model, u[select(i, m)] .<= ū)
    @constraint(jump_model, u[select(i, m)] .>= -ū)
end

@objective(jump_model, Min, objective_exp)

optimize!(jump_model)

set_start_value.(x, value.(x))

X_osqp = [value.(x)[select(i, n)] for i=1:N]
U_osqp = [value.(u)[select(i, m)] for i=1:N-1]

@show objective_value(jump_model)

@benchmark optimize!($jump_model) samples=10 evals=10

In [ ]:
@assert X[1] ≈ x0
@assert value.(x)[select(1, n)] ≈ x0

In [ ]:
using Plots

u_vals = [U[i][1] for i=1:N-1]
u_vals_osqp = [U_osqp[i][1] for i=1:N-1]

plot(1:N-1, u_vals, seriestype=:bar)
plot!(1:N-1, u_vals_osqp, seriestype=:bar)

In [ ]:
objective_value_(X, U) = sum([0.5*X[i]'*Q*X[i] for i=1:N]) + sum([0.5*U[i]'*R*U[i] for i=1:N-1])

@show objective_value_(X, U)
@show objective_value_(X_osqp, U_osqp)